In [1]:
using AbstractGPs
using TemporalGPs

Utilising TemporalGPs.jl to work with AbstractGPs requires minimal modification to the
GP objects from AbstractGPs that you are used to. The primary differences are
1. RegularSpacing is a useful type. It's basically a `range` that's hacked together to
   work nicely with Zygote.jl. At some point, it will hopefully disappear.
2. Call `to_sde` on your AbstractGP object to say "use TemporalGPs.jl to do inference".

This is an example of a very, very noise regression problem.
You would probably be better of using a pseudo-point approximation for this particular
data set, but longer time-series tend to lend themselves less well to nice plots.

Load up the separable kernel from TemporalGPs.

In [2]:
using TemporalGPs: RegularSpacing

Build a GP as per usual, and wrap it inside a TemporalGPs.jl object.

In [3]:
f_raw = GP(5.0, Matern52Kernel());
f = to_sde(f_raw, SArrayStorage(Float64));

Specify a collection of inputs. Must be increasing.

In [4]:
T = 1_000_000;
x = RegularSpacing(0.0, 1e-4, T);

Generate some noisy synthetic data from the GP.

In [5]:
σ²_noise = 5.0;
y = rand(f(x, σ²_noise));

Construct the posterior as per usual.

In [6]:
f_post = posterior(f(x, σ²_noise), y);

Specify some locations at which to make predictions.

In [7]:
T_pr = 1_200_000;
x_pr = RegularSpacing(0.0, 1e-4, T_pr);

Compute the exact posterior marginals at `x_pr`.

In [8]:
f_post_marginals = marginals(f_post(x_pr));
m_post_marginals = mean.(f_post_marginals);
σ_post_marginals = std.(f_post_marginals);

Generate a few posterior samples. Not fantastically-well optimised at present.

In [9]:
f_post_samples = [rand(f_post(x_pr)) for _ in 1:5];

Visualise the posterior. The if block is just to prevent it running in CI.

In [10]:
if get(ENV, "TESTING", "FALSE") == "FALSE"
    using Plots
    plt = plot();
    scatter!(plt, x, y; label="", markersize=0.1, alpha=0.1);
    plot!(plt, f_post(x_pr); ribbon_scale=3.0, label="");
    plot!(x_pr, f_post_samples; color=:red, label="");
    savefig(plt, "posterior.png");
end

"/home/runner/work/TemporalGPs.jl/TemporalGPs.jl/docs/src/examples/exact_time_inference/posterior.png"

<hr />
<h6>Package and system information</h6>
<details>
<summary>Package information (click to expand)</summary>
<pre>
Status &#96;~/work/TemporalGPs.jl/TemporalGPs.jl/examples/exact_time_inference/Project.toml&#96;
  &#91;99985d1d&#93; AbstractGPs v0.5.16
  &#91;98b081ad&#93; Literate v2.14.0
  &#91;91a5bcdd&#93; Plots v1.38.10
  &#91;e155a3c4&#93; TemporalGPs v0.6.3 &#96;/home/runner/work/TemporalGPs.jl/TemporalGPs.jl#61a94af&#96;
</pre>
To reproduce this notebook's package environment, you can
<a href="./Manifest.toml">
download the full Manifest.toml</a>.
</details>
<details>
<summary>System information (click to expand)</summary>
<pre>
Julia Version 1.8.5
Commit 17cfb8e65ea &#40;2023-01-08 06:45 UTC&#41;
Platform Info:
  OS: Linux &#40;x86_64-linux-gnu&#41;
  CPU: 2 × Intel&#40;R&#41; Xeon&#40;R&#41; Platinum 8370C CPU @ 2.80GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 &#40;ORCJIT, icelake-server&#41;
  Threads: 1 on 2 virtual cores
Environment:
  JULIA_DEBUG &#61; Documenter
  JULIA_LOAD_PATH &#61; :/home/runner/.julia/packages/JuliaGPsDocs/e8FS0/src
</pre>
</details>

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*